In [9]:
import logging

import pandas as pd
import numpy as np

import core.finance as cofinanc
import dataflow.core as dtfcore
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint

import research_amp.soccer_prediction.utils as rasoprut

In [2]:
hdbg.init_logger(verbosity=logging.DEBUG)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

DEBUG Effective logging level=10
DEBUG Shut up 116 modules: botocore.loaders, matplotlib.ticker, botocore.args, botocore.retries.adaptive, matplotlib.axis, matplotlib.lines, botocore.endpoint, matplotlib.axes._axes, matplotlib._constrained_layout, botocore.endpoint_provider, matplotlib.dates, matplotlib._layoutgrid, urllib3.poolmanager, botocore.handlers, boto3.resources.factory, urllib3, matplotlib.axes, urllib3.util, botocore.regions, invoke, boto3.resources.base, botocore.waiter, matplotlib.pyplot, boto3, botocore.retryhandler, botocore.configprovider, boto3.resources.action, matplotlib.dviread, boto3.resources, matplotlib.figure, matplotlib.font_manager, botocore.auth, matplotlib.category, botocore.history, botocore.parsers, botocore.utils, matplotlib.text, botocore.compat, fsspec, botocore.awsrequest, matplotlib.mathtext, matplotlib._afm, urllib3.connectionpool, botocore.retries, asyncio, botocore.monitoring, matplotlib.textpath, botocore.discovery, botocore.credentials, botocore.

In [52]:
def preprocess_data(df: pd.DataFrame()) -> pd.DataFrame():
    """
    Preprocess the loaded ISDB dataframe of interest.
        - Filter and select match from seasons starting from 2009.
        - Convert column formats.
        - Add epsilon = 0.5 to scores with value as `0` to avoid log(0).
        - Check for NaN and infinite values and drop the rows.
    
    :param df: Input DataFrame. 
    :return: Preprocessed DataFrame.
    """
    df["season"] = df["Sea"].apply(lambda x: int("20" + str(x)[:2]))
    filtered_df = df[df["season"] >= 2009]
    # Preprocess the dataset.
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
    df.sort_values(by="Date", inplace=True)
    # Covert the categorical columns to category type. 
    categorical_columns = ["HT", "AT"]
    for col in categorical_columns:
        filtered_df[col] = filtered_df[col].astype("category")
    # Adding a small constant to goals to avoid log(0).
    columns = ['AS', 'HS']
    epsilon = 0.0
    for column in columns:    
        filtered_df[column] = filtered_df[column].apply(lambda x: x + epsilon if x == 0 else x)
        # Check if there are any infinite or NaN weights and handle them.
        if filtered_df.isna().sum().sum() > 0:
            _LOG.debug("NaN values found in the data. Removing rows with NaNs.")
            filtered_df.dropna(inplace=True)
        if filtered_df.isin([-np.inf, np.inf]).sum().sum() > 0:
            _LOG.debug("Infinite values found in the data. Removing rows with Infs.")
            filtered_df = filtered_df[~np.isinf(filtered_df.select_dtypes(include=[np.number])).any(1)]
    # Return the preprocessed DataFrame.
    return filtered_df

In [ ]:
# Define the S3 Buckets, dataset path and local directory for download.
bucket = "cryptokaizen-data-test"
dataset_path = "kaizen_ai/soccer_prediction/datasets/OSF_football/"
local_dir = "datasets/OSF_football"
# Download data from S3.
rasoprut.download_data_from_s3(
    bucket_name=bucket, dataset_path=dataset_path, local_path=local_dir
)
# Load the data from S3 into pandas dataframe objects.
dataframes = rasoprut.load_data_to_dataframe(local_path=local_dir)

In [54]:
# Access the dataframes directly from the dictionary.
ISDBv1_df = dataframes.get("ISDBv1_df")
ISDBv2_df = dataframes.get("ISDBv2_df")
# Preprocess the selected dataframe (ISDBv2_df).
preprocessed_df = preprocess_data(ISDBv2_df)
# preprocessed_df.set_index('Date', inplace=True)

In [84]:
preprocessed_df["Lge"].value_counts()

Lge
ENG3    4416
ENG4    4416
ENG2    4416
ENG5    3818
ITA2    3696
SPA2    3696
GER3    3040
ENG1    3040
FRA1    3040
FRA2    3040
SPA1    3040
ITA1    3040
ARG1    3040
BRA1    3039
RUS2    2819
FRA3    2784
JPN2    2690
MEX1    2476
VEN1    2449
GER2    2448
HOL1    2448
GER1    2448
JPN1    2448
USA1    2420
CHL1    2316
BRA2    2280
POR1    2118
ECU1    2052
DZA1    1985
KOR1    1962
MAR1    1920
SWE1    1920
NOR1    1920
ZAF1    1920
RUS1    1920
CHN1    1920
BEL1    1890
GRE1    1811
TUR1    1802
SCO1    1794
ISR1    1630
DNK1    1568
FIN1    1552
TUN1    1546
SCO2    1440
SCO3    1440
AUT1    1440
CHE1    1422
SCO4    1260
AUS1    1110
USA2     809
NZL1     498
Name: count, dtype: int64

In [85]:
SPA1_df = preprocessed_df[preprocessed_df["Lge"] == "SPA1"]
SPA1_df

,Sea,Lge,Date,HT,AT,HS,AS,GD,WDL,season
102914,09-10,SPA1,29/08/2009,Real Madrid,La Coruna,3.0,2.0,1,W,2009
102915,09-10,SPA1,29/08/2009,Zaragoza,Tenerife,1.0,0.0,1,W,2009
102916,09-10,SPA1,30/08/2009,Almeria,Valladolid,0.0,0.0,0,D,2009
102917,09-10,SPA1,30/08/2009,Athletic Bilbao,Espanyol Barcelona,1.0,0.0,1,W,2009
102918,09-10,SPA1,30/08/2009,Malaga,Athletico Madrid,3.0,0.0,3,W,2009
...,...,...,...,...,...,...,...,...,...,...
216920,16-17,SPA1,21/05/2017,Athletico Madrid,Athletic Bilbao,3.0,1.0,2,W,2016
216921,16-17,SPA1,21/05/2017,Valencia,Villarreal,1.0,3.0,-2,L,2016
216922,16-17,SPA1,21/05/2017,Celta de Vigo,Real Sociedad,2.0,2.0,0,D,2016
216923,16-17,SPA1,21/05/2017,FC Barcelona,Eibar,4.0,2.0,2,W,2016


In [89]:
SPA1_df["HT"].value_counts()[:50]

HT
FC Barcelona              152
Malaga                    152
Athletic Bilbao           152
Athletico Madrid          152
Sevilla FC                152
Valencia                  152
Real Madrid               152
Espanyol Barcelona        152
Real Sociedad             133
Villarreal                133
Getafe                    133
Osasuna                   114
La Coruna                 114
Granada                   114
Levante                   114
Celta de Vigo              95
Sporting Gijon             95
Real Betis                 95
Vallecano                  95
Mallorca                   76
Zaragoza                   76
Almeria                    76
Valladolid                 57
Santander                  57
Eibar                      57
Las Palmas                 38
Elche                      38
Xerez                      19
Hercules                   19
Tenerife                   19
Alaves                     19
Leganes                    19
Cordoba                    19
Olympiq

In [61]:
def get_home_team(df, team):
    df = df[df["HT"] == team]
    df["is_home"] = 1
    df = df.rename(columns={"AT" : "opponent", "HS" : "goals_scored", "AS": "goals_scored_by_opponent"})
    df = df.drop(["HT"], axis=1)
    return df.to_dict(orient='records')

def get_away_team(df, team):
    df = df[df["AT"] == team]
    df["is_home"] = 0
    df = df.rename(columns={"HT" : "opponent", "AS" : "goals_scored", "HS": "goals_scored_by_opponent"})
    df = df.drop(["AT"], axis=1)
    # Define the mapping to flip 'W' and 'L'
    flip_mapping = {'W': 'L', 'L': 'W'}
    # Apply the mapping to the 'WDL' column
    df['WDL'] = df['WDL'].replace(flip_mapping)
    df['GD'] = df['GD'].apply(lambda x : -x)
    return df.to_dict(orient='records')

get_away_team(preprocessed_df, 'FC Barcelona')

[{'Sea': '09-10',
  'Lge': 'SPA1',
  'Date': '12/09/2009',
  'opponent': 'Getafe',
  'goals_scored_by_opponent': 0.0,
  'goals_scored': 2.0,
  'GD': 2,
  'WDL': 'W',
  'season': 2009,
  'is_home': 0},
 {'Sea': '09-10',
  'Lge': 'SPA1',
  'Date': '22/09/2009',
  'opponent': 'Santander',
  'goals_scored_by_opponent': 1.0,
  'goals_scored': 4.0,
  'GD': 3,
  'WDL': 'W',
  'season': 2009,
  'is_home': 0},
 {'Sea': '09-10',
  'Lge': 'SPA1',
  'Date': '26/09/2009',
  'opponent': 'Malaga',
  'goals_scored_by_opponent': 0.0,
  'goals_scored': 2.0,
  'GD': 2,
  'WDL': 'W',
  'season': 2009,
  'is_home': 0},
 {'Sea': '09-10',
  'Lge': 'SPA1',
  'Date': '17/10/2009',
  'opponent': 'Valencia',
  'goals_scored_by_opponent': 0.0,
  'goals_scored': 0.0,
  'GD': 0,
  'WDL': 'D',
  'season': 2009,
  'is_home': 0},
 {'Sea': '09-10',
  'Lge': 'SPA1',
  'Date': '31/10/2009',
  'opponent': 'Osasuna',
  'goals_scored_by_opponent': 1.0,
  'goals_scored': 1.0,
  'GD': 0,
  'WDL': 'D',
  'season': 2009,
  'is_

In [93]:
teams = set(preprocessed_df["HT"].to_list() +  preprocessed_df["AT"].to_list())
data = {}
# teams = ['FC Barcelona', 'Real Madrid']
for team in teams:
    data[team] = []
    data[team].extend(get_home_team(SPA1_df, team))
    data[team].extend(get_away_team(SPA1_df, team))
    if len(data[team]) == 0:
        del data[team]

In [94]:
data

{'Real Betis': [{'Sea': '11-12',
   'Lge': 'SPA1',
   'Date': '11/09/2011',
   'opponent': 'Mallorca',
   'goals_scored': 1.0,
   'goals_scored_by_opponent': 0.0,
   'GD': 1,
   'WDL': 'W',
   'season': 2011,
   'is_home': 1},
  {'Sea': '11-12',
   'Lge': 'SPA1',
   'Date': '22/09/2011',
   'opponent': 'Zaragoza',
   'goals_scored': 4.0,
   'goals_scored_by_opponent': 3.0,
   'GD': 1,
   'WDL': 'W',
   'season': 2011,
   'is_home': 1},
  {'Sea': '11-12',
   'Lge': 'SPA1',
   'Date': '02/10/2011',
   'opponent': 'Levante',
   'goals_scored': 0.0,
   'goals_scored_by_opponent': 1.0,
   'GD': -1,
   'WDL': 'L',
   'season': 2011,
   'is_home': 1},
  {'Sea': '11-12',
   'Lge': 'SPA1',
   'Date': '23/10/2011',
   'opponent': 'Vallecano',
   'goals_scored': 0.0,
   'goals_scored_by_opponent': 2.0,
   'GD': -2,
   'WDL': 'L',
   'season': 2011,
   'is_home': 1},
  {'Sea': '11-12',
   'Lge': 'SPA1',
   'Date': '05/11/2011',
   'opponent': 'Malaga',
   'goals_scored': 0.0,
   'goals_scored_by_o

In [97]:
dfs = []

for key, inner_list in data.items():
    df_inner = pd.DataFrame(inner_list)
    df_inner['outer_key'] = key  # Add outer key as a column
    dfs.append(df_inner)

# Concatenate all DataFrames
df_concat = pd.concat(dfs, ignore_index=True)

df_concat['Date'] = pd.to_datetime(df_concat['Date'], format='mixed')

# Pivot the DataFrame to have 'date' as index and 'outer_key' as columns
df_pivot = df_concat.pivot_table(index='Date', columns='outer_key', aggfunc='first')

# Sort the columns to ensure the outer keys are grouped together
df_pivot = df_pivot.sort_index(axis=1, level=0)

df_pivot

GD                                                                                                                                                                                                                                                                                                                                              Lge                                                                                                                                                                                                                                                                                                                                              Sea                                                                                                                                                                                                                                                                                                                  \
outer_key  Alaves Almeria Athletic Bilbao Athletico Madrid Celta de Vigo Cordoba Eibar Elche Espanyol Barcelona FC Barcelona Getafe Granada Hercules La Coruna Las Palmas Leganes Levante Malaga Mallorca Osasuna Real Betis Real Madrid Real Sociedad Santander Sevilla FC Sporting Gijon Tenerife Valencia Valladolid Vallecano Villarreal Xerez Zaragoza Alaves Almeria Athletic Bilbao Athletico Madrid Celta de Vigo Cordoba Eibar Elche Espanyol Barcelona FC Barcelona Getafe Granada Hercules La Coruna Las Palmas Leganes Levante Malaga Mallorca Osasuna Real Betis Real Madrid Real Sociedad Santander Sevilla FC Sporting Gijon Tenerife Valencia Valladolid Vallecano Villarreal Xerez Zaragoza Alaves Almeria Athletic Bilbao Athletico Madrid Celta de Vigo Cordoba Eibar Elche Espanyol Barcelona FC Barcelona Getafe Granada Hercules La Coruna Las Palmas Leganes Levante Malaga Mallorca Osasuna Real Betis Real Madrid Real Sociedad Santander Sevilla FC Sporting Gijon Tenerife Valencia Valladolid Vallecano   
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
2009-01-11    NaN    -1.0             NaN              NaN           NaN     NaN   NaN   NaN                0.0          NaN    NaN     NaN      NaN       0.0        NaN     NaN     NaN   -1.0      1.0     NaN        NaN         NaN           NaN      -1.0        NaN            0.0     -5.0      1.0        0.0       NaN        5.0   NaN      1.0    NaN    SPA1             NaN              NaN           NaN     NaN   NaN   NaN               SPA1          NaN    NaN     NaN      NaN      SPA1        NaN     NaN     NaN   SPA1     SPA1     NaN        NaN         NaN           NaN      SPA1        NaN           SPA1     SPA1     SPA1       SPA1       NaN       SPA1   NaN     SPA1    NaN   09-10             NaN              NaN           NaN     NaN   NaN   NaN              09-10          NaN    NaN     NaN      NaN     09-10        NaN     NaN     NaN  09-10    09-10     NaN        NaN         NaN           NaN     09-10        NaN          09-10    09-10    09-10      09-10       NaN   
2009-02-12    NaN  

In [23]:
# `nid` is short for "node id"
nid = "df_data_source"
df_data_source = dtfcore.DfDataSource(nid, preprocessed_df)

In [24]:
df_out_fit = df_data_source.fit()["df_out"]
_LOG.debug(hpandas.df_to_str(df_out_fit))

,Sea,Lge,HT,AT,HS,AS,GD,WDL,season
2009-01-31 00:00:00,09-10,ECU1,Manta FC,Barcelona SC,0.5,0.5,0,D,2009
2009-01-31 00:00:00,09-10,CHL1,Curico Unido,Colo Colo,2.0,2.0,0,D,2009
2009-01-31 00:00:00,09-10,CHL1,Everton CD,Union Espanola,0.5,1.0,-1,L,2009
,...,...,...,...,...,...,...,...,...
2017-06-27 00:00:00,16-17,ARG1,Estudiantes,Quilmes AC,1.0,0.5,1,W,2016
2017-06-28 00:00:00,16-17,ARG1,Talleres Cordoba,San Lorenzo,1.0,1.0,0,D,2016
2017-06-28 00:00:00,16-17,ARG1,Independiente,CA Lanus,1.0,1.0,0,D,2016


DEBUG None


In [ ]:
"""
  is_home              GS
Manta FC

"""

In [25]:
def unravel_df(df: pd.DataFrame()) -> pd.DataFrame():
    """    
    Unravel the dataset by creating two entries for each row as team-opponent
    pair.
    
    :param df: Input dataframe.
    :return: unraveled dataframe.
    """
    # Create entry for home team `HT`.
    home_df = df[["Sea", "Lge", "HT", "AT", "HS"]].copy()
    home_df.rename(
                columns={"HT": "team", "AT": "opponent", "HS": "goals"}, inplace=True
                  )
    home_df["is_home"] = 1
    # Create entry for away team `AT`.
    away_df = df[["Sea", "Lge", "HT", "AT", "AS"]].copy()
    away_df.rename(
                columns={"AT": "team", "HT": "opponent", "AS": "goals"}, inplace=True
                  )
    away_df["is_home"] = 0
    # Concatenate the two splits.
    unraveled_df = pd.concat([home_df, away_df], ignore_index=True)
    # return the unraveled dataframe.
    return unraveled_df

In [40]:
# Prepare a DataFlow transformer node to compute percentage returns
#  and only return percentage returns.
nid = "unravel_dataset"
unravel_node_v2 = dtfcore.GroupedColDfToDfTransformer(
    nid,
    transformer_func=unravel_df,
    in_col_groups=[('Sea', 'Lge', 'HT', 'AT', 'HS', 'AS', 'GD', 'WDL', 'season')],
    out_col_group=(),
)

In [41]:
unravel_node_v2.fit(preprocessed_df)["df_out"]

AssertionError: 
################################################################################
* Failed assertion *
'('Sea', 'Lge', 'HT', 'AT', 'HS', 'AS', 'GD', 'WDL', 'season')' in 'Index(['Sea', 'Lge', 'HT', 'AT', 'HS', 'AS', 'GD', 'WDL', 'season'], dtype='object')'
################################################################################
